In [1]:
import pathlib
import datetime
import json

DATA_DIR = pathlib.Path('/home/michel/projects/desinfo/political-ads-scraper/data')

In [4]:
ads_by_date = {}

for filename in sorted(list((DATA_DIR / 'twitter').iterdir())):
    if filename.stem == 'graph_nb_ads_EU_twitter':
        continue

    fetch_datetime = datetime.datetime.strptime(
        filename.stem,
        'twitter-ads_EU_%Y-%m-%d_%H-%M-%S',
    )

    with open(filename, 'r') as f:
        ads = json.load(f)

    ads_by_date[fetch_datetime.strftime("%Y-%m-%d")] = ads


In [5]:
ads_by_date.keys()

dict_keys(['2019-06-07', '2019-06-08', '2019-06-09', '2019-06-10', '2019-06-11', '2019-06-12', '2019-06-13', '2019-06-14', '2019-06-15', '2019-06-16', '2019-06-17', '2019-06-18'])

In [6]:
ads_by_date['2019-06-10'].keys()

dict_keys(['advertisers_data', 'advertisers_tweets'])

## Advertisers

In [7]:
adv_lists = [
    tuple([
        adv['id_str']
        for adv in ads_by_date[date]['advertisers_data']
    ])
    for date in ads_by_date.keys()
]

In [8]:
assert len(set(adv_lists)) == 1
adv_ids = adv_lists[0]

## Tweets

In [26]:
for adv_id in adv_ids:
    for date_str in list(ads_by_date.keys())[:-1]:
        date = datetime.date(*map(int, date_str.split('-')))
        date_tomorrow = date + datetime.timedelta(1)

        tweet_ids_1 = set([
            tweet['basic_data']['tweetId']
            for tweet in ads_by_date[date_str]['advertisers_tweets'][adv_id]
        ])
        tweet_ids_2 = set([
            tweet['basic_data']['tweetId']
            for tweet in ads_by_date[str(date_tomorrow)]['advertisers_tweets'][adv_id]
        ])

        if not (tweet_ids_1 <= tweet_ids_2):
            adv_screen_name = [
                adv_data['screen_name']
                for adv_data in ads_by_date['2019-06-12']['advertisers_data']
                if adv_data['id_str'] == adv_id
            ][0]
            
            print('Tweets were removed between {} and {} for advertiser {} ({})'.format(
                date_str, date_tomorrow, adv_screen_name, adv_id
            ))
            for tweet_id in tweet_ids_1 - tweet_ids_2:
                tweet = [
                    tweet
                    for tweet in ads_by_date[date_str]['advertisers_tweets'][adv_id]
                    if tweet['basic_data']['tweetId'] == tweet_id
                ][0]
                print('    https://twitter.com/{}/status/{}'.format(adv_screen_name, tweet_id))
                if tweet['additional_data']:
                    print('    {}'.format(tweet['additional_data']['full_text']))
                print()


Tweets were removed between 2019-06-10 and 2019-06-11 for advertiser Der_BDI (57043880)
    https://twitter.com/Der_BDI/status/1135824679208443904

    https://twitter.com/Der_BDI/status/1135846996542152704
    Die Tradition bleibt erhalten! @peteraltmeier bereitet sich auf die Rede vor! #TDI19 #WhichEurope @BMWi_Bund 📢🇪🇺👏 https://t.co/iGQYPgcJK0

    https://twitter.com/Der_BDI/status/1135571971251613696
    Starke Worte von @PierreGattaz beim #TDI19! #WhichEurope https://t.co/Myzox5UmsN

    https://twitter.com/Der_BDI/status/1135548842999472129
    Einen wunderschönen Nachmittag aus dem Funkhaus #Berlin. Früher Sendestation des DDR-Staatsradios, heute Veranstaltungsort eines Festivals der Meinungsfreiheit! 😉 Wir sind vor Ort und twittern, was das Zeug hält! #TDI19 #WhichEurope 🇪🇺🚀 https://t.co/9OWT5FAXRV

    https://twitter.com/Der_BDI/status/1135824358608375809
    Die #Bundeskanzlerin: "Danke dem BDI und Ihnen allen, dass Sie für Europawahl geworben haben. Damit haben Sie dazu be